### Question Answer Application
The goal of Question Answering is to find the answer to a question given a question and an accompanying context. The predicted answer will be either a span of text from the context or an empty string (indicating the question cannot be answered from the context).

In [ ]:
#!pip install datasets
#!pip install apache_beam
#!pip install simpletransformers

In [1]:
#from google.colab import drive
import sys
from datasets import load_dataset
import json
from time import time
import glob

In [2]:
def jsonfile_reader(path):
	dataset = []
	for file in glob.glob(path, recursive=True):
		with open(file) as json_file:
			dataset.append(json.load(json_file))
	return dataset

In [3]:
#drive.mount('/drive')
#!ls ../drive/MyDrive/IKT450/
#sys.path.append('/drive/My Drive/Medium')

local = False
if local:
	dataset = jsonfile_reader('file.json')
else:
	dataset_train = load_dataset('natural_questions', split='train')
	dataset_validation = load_dataset('natural_questions', split='validation')

print("train: ",len(dataset_train))
print("validation: ",len(dataset_validation))


ERROR: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/apache_beam/io/gcp/bigquery.py", line 341, in <module>
    import google.cloud.bigquery_storage_v1 as bq_storage
ModuleNotFoundError: No module named 'google.cloud'
Using custom data configuration default
Reusing dataset natural_questions (/home/runeal17/.cache/huggingface/datasets/natural_questions/default/0.0.2/19bc04755018a3ad02ee74f7045cde4ba9b4162cb64450a87030ab786b123b76)
Using custom data configuration default
Reusing dataset natural_questions (/home/runeal17/.cache/huggingface/datasets/natural_questions/default/0.0.2/19bc04755018a3ad02ee74f7045cde4ba9b4162cb64450a87030ab786b123b76)


train:  307373
validation:  7830


In [4]:
def remove_html(data):
	long_answer_temp = []
	for i, each in enumerate(data['is_html']):
		if each == 0:
			long_answer_temp.append(data['token'][i])
	return long_answer_temp


def zupp(start_token, tokens):
	temp = []
	for each in range(start_token - 3, start_token + 4):
		temp.append(tokens[each])
	return ' '.join(map(str, temp))


def create_data(dataset):
	total = []
	for each in dataset:
		tokens = each['document']['tokens']['token']
		answer_text = ''
		search_string = ''
		for i in range(len(each['annotations']['short_answers'])-1):
			try:
				if each['annotations']['short_answers'][i]['start_token'][0]:
					start_byte = each['annotations']['short_answers'][i]['start_byte'][0]
					search_string = zupp(start_byte, tokens)
					answer_text = each['annotations']['short_answers'][i]['text'][0]
					break
				else:
					continue
			except:
				continue

		
		context = ' '.join(map(str, remove_html(each['document']['tokens'])))

		total.append({
			"context": context,
			"qas": [
				{
					"id": each['annotations']['id'][0],
					"is_impossible": False,
					"question": each['question']['text'],
					"answers": [
						{
							"text": answer_text,
							"answer_start": context.find(search_string) + search_string.find(answer_text),
						}
					],
				}
			],
		})
	return total
print("Done")

Done


In [5]:
def create_data_predict(dataset):
	total = []
	for each in dataset:
		tokens = each['document']['tokens']['token']
		answer_text = ''
		search_string = ''
		for i in range(len(each['annotations']['short_answers'])-1):
			try:
				if each['annotations']['short_answers'][i]['start_token'][0]:
					start_byte = each['annotations']['short_answers'][i]['start_byte'][0]
					search_string = zupp(start_byte, tokens)
					answer_text = each['annotations']['short_answers'][i]['text'][0]
					break
				else:
					continue
			except:
				continue

		context = ' '.join(map(str, remove_html(each['document']['tokens'])))

		total.append({
			"context": context,
			"qas": [
				{
					"question": each['question']['text'],
					"id": each['annotations']['id'][0],
				}
			],
		})
	return total
print("Done")


Done


In [70]:
# dataset sizes

train_size = 2
test_size = 1000
validation_size = 2

In [38]:
# Create Train set

start = time()
train_temp = []
for i in range(train_size):
	train_temp.append(dataset_train[i])
train = create_data(train_temp)
stop = time()

print("Create train: ", (stop - start),"seconds")

Create train:  0.011138439178466797 seconds


In [39]:
# Create Test set

start = time()
test_temp = []
for i in range(test_size):
	test_temp.append(dataset_validation[i])
test = create_data(test_temp)
stop = time()

print("create test: ", (stop - start),"seconds")

create test:  9.730314493179321 seconds


In [72]:
# Create Validation set

start = time()
vali_temp = []
for i in range(validation_size):
	vali_temp.append(dataset_train[i])
    #append(dataset_validation[i+test_size])
validation = create_data_predict(vali_temp)
stop = time()

print("Create validation: ", (stop - start),"seconds")

Create validation:  0.016514301300048828 seconds


In [41]:
#validation[0]

In [60]:
print("Train length: ",len(train))
print("Test length: ",len(test))
print("validation length: ",len(validation))

Train length:  2
Test length:  1000
validation length:  5


In [43]:
# Original train from github repo

#with open(r"train.json", "r") as read_file:
 #   train = json.load(read_file)

    #with open(r"test.json", "r") as read_file:
#    test = json.load(read_file)

In [44]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [28]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [29]:
# Configure the model 
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5


In [50]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs2/{model_type}",
    "best_model_dir": f"outputs2/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 30,
    "evaluate_during_training_steps": 10,
    "wandb_project": "Question Answer Application 3",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    #"n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [51]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [52]:
### Remove output folder
#!rm -rf outputs

In [53]:
# Train the model
start = time()
model.train_model(train, eval_data=test)
stop = time()

print("Training: ", (stop - start))


add example index and unique id: 100%|██████████| 2/2 [00:00<00:00, 26800.66it/s]


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

correct,▁▁▂▅█
eval_loss,██▅▃▁
global_step,▁▃▅▆█
incorrect,▁▁▁▁▁
similar,██▇▄▁
train_loss,█▇█▃▁
correct,329
eval_loss,-0.39752
global_step,5
incorrect,0
similar,671


Running Epoch 0 of 100:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/360 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/360 [00:00<?, ?it/s]

Running Epoch 31 of 100:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/360 [00:00<?, ?it/s]

Running Epoch 32 of 100:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [54]:
# Evaluate the model
result, texts = model.eval_model(test)

result
# 'texts' is an array containing all the question id's

Running Evaluation:   0%|          | 0/360 [00:00<?, ?it/s]

{'correct': 1000, 'similar': 0, 'incorrect': 0, 'eval_loss': -5.678515625}

In [ ]:
print(texts)

# What is the result over here ?







In [67]:
# Make predictions with the model
to_predict = [
    {'context': "Red Bull - wikipedia Red Bull Jump to : navigation , search This article is about the energy drink . For the company , see Red Bull GmbH . For other uses , see Red Bull ( disambiguation ) . Red Bull Type Energy drink Distributor Red Bull GmbH Country of origin Thailand Introduced 1976 ( as Krating Daeng ) Color Amber Ingredients Caffeine , taurine , sucrose and glucose , B - group vitamins , and alpine spring water Variants Original Sugarfree Total Zero Red / Cranberry Edition Blue / Blueberry Edition Yellow / Tropical Edition Orange / Mandarine Edition Lime / Silver / Green Edition Spring Edition Green / Kiwi - Apple Edition Winter Edition Summer Edition The `` BULL '' Edition Red Bull G - Drive Edition F1 Edition Orange Edition Zero Cherry Edition Zero Lime Edition Sugarfree Purple Edition Sugarfree ( Acai berry ) Purple Edition Related products Krating Daeng , Red Bull Cola , Red Bull Energy Shot , Red Bull Sugar - Free , Red Bull Total Zero Website www.redbull.com Red Bull is an energy drink sold by Austrian company Red Bull GmbH , created in 1987 . Red Bull has the highest market share of any energy drink in the world , with 6.06 billion cans sold in 2016 . Austrian entrepreneur Dietrich Mateschitz was inspired by an existing energy drink named Krating Daeng , which was first introduced and sold in Thailand by Chaleo Yoovidhya . He took this idea , modified the ingredients to suit the tastes of Westerners , and , in partnership with Chaleo , founded Red Bull GmbH in 1987 in Chakkapong , Thailand . In Thai , daeng means red , and krating is the word used for a large bovine animal called a gaur in formal English . In colloquial English , a gaur is more commonly called a bull . Red Bull is sold in a tall and slim blue - silver can , while Krating Daeng is in a shorter gold can . The two are different products , produced separately . The Red Bull company slogan is `` Red Bull gives you wings '' . Rather than following a traditional approach to mass marketing , Red Bull has generated awareness and created a seductive ' brand myth ' through proprietary extreme sport event series such as Red Bull Cliff Diving World Series , Red Bull Air Race , Red Bull Crashed Ice and stand - out stunts such as the Stratos space diving project . Red Bull 's marketing arsenal also includes multiple sports team ownerships ( Flying Bulls ( actually owned by Dietrich Mateschitz ) , RB Leipzig , FC Red Bull Salzburg , Red Bull Brasil , New York Red Bulls , Red Bull Racing , Scuderia Toro Rosso ) , celebrity endorsements , and music , through its record label Red Bull Records . Energy drinks have been associated with health risks , such as masking the effects of intoxication when consumed with alcohol , and excessive or repeated consumption can lead to cardiac and psychiatric conditions . However , the European Food Safety Authority ( EFSA ) concluded that an adequate consumption of Red Bull and other popular energy drinks is safe and that the amount of caffeine in standard Red Bull cans is unlikely to interact adversely with other typical constituents of energy drinks or with alcohol . Energy drinks have the effects that caffeine and sugar provide , but there is no distinct evidence that the wide variety of other ingredients has any effect . Contents ( hide ) 1 History 2 Ingredients 3 Health effects 3.1 Caffeine content 4 Market approval and legal status 5 Litigation 6 Advertising 6.1 Red Bull House of Art 6.2 Sports sponsorships and acquisitions 6.3 Sponsorships 6.4 Endorsements 6.5 Events 6.6 Locations 7 See also 8 References 9 External links History ( edit ) Red Bull In front of the Potala Palace , Tibet : a model of Red Bull in Chinese version is displayed . Red Bull car In 1976 , Chaleo Yoovidhya introduced a drink called Krating Daeng in Thailand , which means `` red gaur '' in English . It was popular among Thai truck drivers and labourers . While working for German manufacturer Blendax ( later acquired by Procter & Gamble ) in 1982 , Dietrich Mateschitz travelled to Thailand and met Chaleo , owner of T.C. Pharmaceutical . During his visit , Mateschitz discovered that Krating Daeng helped cure his jet lag . In 1984 , Mateschitz co-founded Red Bull GmbH with Yoovidhya and turned it into an international brand . Each partner invested US $500,000 of savings to found the company . Yoovidhya and Mateschitz each held a 49 percent share of the new company . They gave the remaining two percent to Yoovidhya 's son , Chalerm , but it was agreed that Mateschitz would run the company . The product was launched in Austria in 1987 . In Thailand , energy drinks are most popular with blue - collar workers . Red Bull re-positioned the drink as a trendy , upscale drink , first introducing it at Austrian ski resorts . Pricing was a key differentiator , with Red Bull positioned as a premium drink and Krating Daeng as a lower cost item . In many countries both drinks are available , dominating both ends of the price spectrum . In 1992 , the product expanded to international markets : Hungary and Slovenia . It entered the United States via California in 1997 and the Middle East in 2000 . In 2008 , Forbes magazine listed both Chaleo and Mateschitz as the 250th richest people in the world with an estimated net worth of US $4 billion . Red Bull GmbH is headquartered in Fuschl am See , an Austrian village of about 1,500 inhabitants near Salzburg . The company is 51 percent controlled by the Yoovidhya family who , for technical reasons , own the trademark in Europe and the US . In 1995 , Krating Daeng authorized its drink . labelled as Red Bull , to be sold in China . Since 2014 , the Austrian Red Bull ( carbonated ) has also been exported to China . This has created confusion since both drinks use the same brand name , in both English and Chinese . Similarly , in Southeast Asia , Red Bull and Krating Daeng are often confused as both use the Red Bull name in their packaging , although they are two separate products aimed at different markets . The main difference is that Red Bull comes in a tall blue and silver can while the Thailand Red Bull , or Krating Daeng , is in a smaller gold can . The two drinks also differ in terms of taste -- - Red Bull has less sugar and is carbonated . The flavouring used for Red Bull is still produced in Bangkok and exported worldwide . Ingredients ( edit ) Red Bull contains caffeine , taurine , B vitamins ( B3 , B5 , B6 , B12 ) , sucrose , and glucose . To produce Red Bull Sugarfree , sugars sucrose and glucose have been replaced by the sweeteners acesulfame K and aspartame / sucralose . Health effects ( edit ) Further information : Energy drinks § Effects Claims about the drink 's effects and performance have been challenged on various occasions , with the UK 's Advertising Standards Authority imposing advertising restrictions in 2001 in response to complaints recorded as early as 1997 . The latest dietary guidelines from the U.S. suggest that moderate caffeine intake may not only be safe , but also healthy . Studies from the U.S. Department of Agriculture and the U.S. Department of Health and Human Services show that daily consumption of the amount of caffeine contained in three to five cups of coffee is not only safe , but also appears to reduce the risk of Type 2 diabetes and cardiovascular disease in adults . Caffeine may even protect against Parkinson 's disease , the evidence suggests . Energy drinks have the effects that caffeine and sugar provide , but experts still argue about the possible effects of the other ingredients . Most of the effects of energy drinks on cognitive performance , such as increased attention and reaction speed , are primarily due to the presence of caffeine . Still there is evidence that energy drinks can increase mental and athletic performance . Performance of prolonged driving , for example is increased significantly after consumption of Red Bull . Other tests for physical performance showed results such as increased endurance and power . Red Bull energy drink significantly increased upper body muscle endurance during repeated Wingate tests in young healthy adults . Excessive or repeated consumption of energy drinks can lead to cardiac and psychiatric conditions . The European Food Safety Authority ( EFSA ) concluded that exposure to taurine and glucuronolactone at the levels presently used in energy drinks is no safety concern . In a separate analysis , they also concluded that there is insufficient evidence to support a number of commercial health claims about taurine . A review published in 2008 found no documented reports of negative or positive health effects associated with the amount of taurine used in energy drinks , including Red Bull . The review also states that though the caffeine and sugar levels in the energy drink are comparable to those present in coffee and fruit juice respectively , these levels have been shown to cause adverse health effects . In its scientific opinion on the safety of caffeine of 2015 the EFSA concludes , that `` Consumption of other constituents of energy drinks at concentrations commonly present in such beverages would not affect the safety of single doses of caffeine up to 200 mg . '' Also the consumption of alcohol , leading to a blood alcohol content of about 0.08 % , would , according to the EFSA , not affect the safety of single doses of caffeine up to 200 mg . Up to these levels of intake , caffeine is unlikely to mask the subjective perception of alcohol intoxication . Habitual use of caffeine up to 400 mg per day does not . According to SAMHSA , the number of people hospitalized due to energy drinks doubled between 2007 and 2014 in the US . Caffeine content ( edit ) This article needs additional citations for verification . Please help improve this article by adding citations to reliable sources . Unsourced material may be challenged and removed . ( March 2017 ) ( Learn how and when to remove this template message ) The caffeine content of a single can of Red Bull is 80 mg / 250 ml ( 32 mg / 100 ml ) . This is about the same as one cup of coffee , or slightly less depending on the brewing method . The actual caffeine level in Red Bull can vary depending on country , as some countries have legal restrictions on how much caffeine is allowed in drinks . As is the case with other caffeinated beverages , Red Bull drinkers may experience adverse effects as a result of overuse . Excessive consumption may induce mild to moderate euphoria primarily caused by stimulant properties of caffeine and may also induce agitation , anxiety , irritability and insomnia . The general population of healthy adults is not at risk for potential adverse effects from caffeine if they limit their consumption to 400 mg per day , which is the equivalent of 5 standard cans , with one can being 250ml in volume . Consumption of a single energy drink will not lead to excessive caffeine intake . Adverse effects associated with caffeine consumption in amounts greater than 400 mg include nervousness , irritability , sleeplessness , increased urination , abnormal heart rhythms ( arrhythmia ) , and dyspepsia . Consumption also has been known to cause pupil dilation when taken with certain antidepressants or SSRIs . Caffeine dosage is not required to be on the product label for food in the United States , unlike drugs , but some advocates are urging the FDA to change this practice . Market approval and legal status ( edit ) Authorities in France , Denmark , and Norway initially kept Red Bull from being sold domestically . However , as of 2014 , it is on sale in all 28 member states of the European Union and in more than 167 countries around the world . The French food safety agency was concerned about taurine . A Red Bull drink that did not contain taurine was introduced . The French refusal of market approval was challenged by the European Commission , and partially upheld by the European Court of Justice in 2004 . The French food safety agency relented in 2008 , because it was unable to prove a definite health risk , taurine - related or not . In 2012 , the Kuwaiti Ministry of Commerce banned Red Bull for people under 16 after it had allegedly caused heart attacks for a 16 - year - old and a 21 - year - old national squash team player . Litigation ( edit ) In 2013 , Red Bull told the Redwell Brewery , a Norfolk micro brewery , to change its name or face legal action , because it sounded too similar to Red Bull . The eight - man brewery in Norwich was told its name could `` confuse '' customers and `` tarnish '' its trademark . The two companies reached a settlement permitting Redwell to continue using its name . In 2014 , Red Bull faced a US $13 million settlement addressing two consumer class action lawsuits in the United States District Court for the Southern District of New York . The listed plaintiffs were Benjamin Careathers , David Wolf , and Miguel Almarez who sued the company for Red Bull 's marketing and labeling , claiming that the company violated `` express warranty and unjust enrichment '' since their products were first launched . The court hearing took place the morning of 1 May 2015 in an attempt to determine approval for the settlement . Customers that submitted claims then have the opportunity to receive a US $10 cash reimbursement or product option of US $15 in Red Bull products within 150 days of the approved appeals . Advertising ( edit ) It has been suggested that this article be split into a new article titled Red Bull GmbH . ( Discuss ) ( March 2016 ) Red Bull 's TAH - 1F Cobra helicopter assembled by Chuck Aaron , owned by the Flying Bulls Red Bull 's colours on De Havilland Aviation 's ex-Royal Navy de Havilland Sea Vixen . Max Verstappen in the 2017 Formula 1 car of the Red Bull Racing Team Red Bull 's slogan was `` Gives you wings '' until they were hit with a $13 million class - action lawsuit in late 2014 . This was Red Bull 's slogan for over 20 years until one consumer , Benjamin Careathers , decided to sue the company for ' false advertising ' , arguing that after 10 years drinking Red Bull he neither had wings nor any enhanced athletic or intellectual performance . Red Bull 's international marketing campaign targets young men mostly with extreme sports . These range from mountain biking , BMX , motocross , windsurfing , snowboarding , skateboarding , kayaking , rowing , wakeboarding , cliff - diving , parkour , surfing , skating , freestyle motocross , rallycross , Formula 1 racing , NASCAR racing , to breakdancing . Red Bull uses music and videogames , and has enlisted celebrities , such as Eminem ( sponsoring the Red Bull `` EmSee Battle Rap championships '' ) . It hosts events like art shows and the `` Red Bull Flugtag '' ( German for `` flight day '' or `` flying day '' ) . Red Bull owns football teams , with clubs in Austria , Germany , the United States and Brazil featuring the Red Bull trademark in their names . By associating the drink 's image with these activities , the company seeks to promote a `` cool '' public image and raise brand power . The energy drink has created a market for over 150 related types of products . In the PlayStation 3 's social gaming platform , PlayStation Home , Red Bull developed its own in - game island , specifically advertising its energy drink and the Red Bull Air Race event ( for which the space is named ) released in January 2009 . In late November 2009 , Red Bull produced two new spaces , the Red Bull Illume space , and the Red Bull Beach space featuring the Red Bull Flugtag , both released on the same day . In January 2012 , Red Bull released its first personal space called the `` Red Bull House of Skate '' featuring an indoor skate park . On the Apple App store Red Bull also has a few apps which include : RedBull.com , Red Bull TV , The Red Bulletin , RBMA Radio , Red Bull iFunk , Wings For Life - Selfie Run . As well as some games on the app store which are : Bike Unchained , Red Bull Kart Fighter 3 , Red Bull Air Race The Game , Red Bull Racers . In 2010 , the company enlisted Adrian Newey to design a prototype racing car , the Red Bull X2010 , for the videogame Gran Turismo 5 . Red Bull House of Art ( edit ) The Red Bull House of Art is an art fellowship program that was launched by Red Bull . The program is held in multiple cities , most notably the Red Bull House of Art programs in Detroit , Michigan and São Paulo , Brazil , and the program typically consists of a three - month period during which time six to eight participants will create new artwork with the intent to display it in a final exhibition . During the fellowship the artists receive unlimited access to the galleries and a stipend for art supplies . The program has received criticism as being an `` elaborate advertising scheme '' , as some of the artwork has been used in Red Bull advertising campaigns . Sports sponsorships and acquisitions ( edit ) RB Leipzig : a football club currently playing in the German 1 . Bundesliga New York Red Bulls ( officially Red Bull New York ) : a football franchise competing in Major League Soccer of the United States Red Bull Racing -- a Formula One team based in Milton Keynes , England , World Constructor 's Championship winner of 2010 , 2011 , 2012 and 2013 , and produced the car for the 2010 , 2011 , 2012 and 2013 F1 World Champion Sebastian Vettel . Red Bull Sailing Team : an Extreme Sailing Series racing team . Scuderia Toro Rosso ( Italian translation of Team Red Bull ) : Another Formula One team based in Faenza , Italy . Team Red Bull : a Toyota racing team competing in the U.S. - based stock car racing competition NASCAR Red Bull Racing Team competed in NASCAR beginning in 2007 . The team ceased operations after the Sprint Cup Series ' 2011 season . FC Red Bull Salzburg : an Austrian football club based in Salzburg and competing in the Austrian Bundesliga EC Red Bull Salzburg : a member of the Austrian Hockey League based in Salzburg , Austria EHC Red Bull München : a member of the Deutsche Eishockey Liga based in Munich , Germany Red Bull Brasil : a football club currently playing in the Brazilian Campeonato Paulista based in Campinas , Brazil Red Bull Holden Racing Team - Run by Triple Eight Race Engineering -- since 2013 , Red Bull has sponsored the V8 Supercars team . Red Bull Racing Brasil , a Stock Car Brasil team , since 2007 . Red Bull KTM Factory Racing , a team competing in MotoGP . Red Bull Ghana : was a Ghanaian professional football club located in Sogakope that played in the Poly Tank Division One League . Škoda Motorsport rally team Red Bulls , a League of Legends team currently playing in the European League of Legends Challenger Series Esports team OG Dota 2 . Sponsorships ( edit ) Audi A4 DTM , which won the Manufacturers ' championship in 2004 The Citroën rally car , which won the manufacturers ' title in 2008 , 2009 and 2010 . A Red Bull sponsored track racing bike A Sauber C15 from 1995 in display at Red Bull Hangar - 7 A VW Touareg during the Dakar Rally , which won the event in 2009 , 2010 and 2011 A KTM , which won the motocross world championship in 2010 , 2011 , 2012 and 2013 Abt Sportsline Arrows F1 team from 2001 to 2002 Audi Sport for the A4 DTM Batalla de los Gallos Cacá Bueno 's speed record Vectra Carlin Motorsport Dallara - Renault Citroën World Rally Team ( 2008 - 2012 ) EKS RX Enrique Cedeño Martinez ( xPeke ) , professional League of Legends player Ford Fiesta WRC Hansen Motorsport rallycross team Hyundai Genesis Coupe Pikes Peak edition Hyundai Veloster rallycross car Iveco Dakar rally trucks JMB Racing Maserati MC12 GT KTM road , MXGP and Dakar rally bikes Red Bull Ajo Motorsport Red Bull Youth America 's Cup Repsol Honda Team OG Dota 2 eSports team Oracle Team USA Olsbergs MSE Ford rallycross team Peugeot 208 T16 Peugeot 2008 DKR ( 2015 ) Sauber F1 team ( 1995 -- 2004 ) SEAT Sport WTCC team Škoda Fabia S2000 Red Bull Honda World Superbike Team Volkswagen Passat DARPA Grand Challenge edition Andrew Dickson , pro BMX rider Volkswagen Polo R WRC Volkswagen Race Touareg Dakar edition Volkswagen V10 TDI Touaregs Pikes Peak edition Endorsements ( edit ) In 2009 , Red Bull added mainstream sport athletes to its roster of endorsements , which also includes persons involved in non-sporting activities . Reggie Bush , US NFL athlete announced at the 2009 Super Bowl . Ryan Dungey American Supercross , Motocross rider for Red Bull KTM Factory Racing . Marvin Musquin French Supercross , Motocross rider for Red Bull KTM Factory Racing . Dani Pedrosa , Marc Márquez , John Reynolds , Andrea Dovizioso , Casey Stoner , and Nicky Hayden Red Bull Junior Team , a program which sponsors young drivers competing in single seater racing . Since 2001 many young drivers developed careers in professional motorsport , with 13 participants having competed in Formula One , including quadruple F1 world champion Sebastian Vettel , Daniel Ricciardo , and Max Verstappen Skijumpers Gregor Schlierenzauer , Thomas Morgenstern , Adam Małysz , and Andreas Goldberger , snowboarders Mark McMorris , Torah Bright and Nathan Johnstone , and skiers Kaya Turski , Lindsey Vonn , Axel Lund Svindal and Alexis Pinturault . Dion Phaneuf NHL defenseman , currently with the Ottawa Senators . Young professional golfer Rickie Fowler . Racing driver Andrew Jordan . Drift racing driver `` Mad '' Mike Whiddett . Professional chess player Hikaru Nakamura . Super Smash Bros. Melee player William `` Leffen '' Hjelte Street Fighter players Daigo Umehara , Masato `` Bonchan '' Takahashi , Darryl `` Snake Eyez '' Lewis and Olivier `` Luffy '' Hay . Events ( edit ) Red Bull Flugtag Stockholm 2010 Red Bull Flugtag Stockholm 2010 Red Bull sports events Red Bull 400 Red Bull Battle Grounds : An eSports competition in popular video games Dota 2 and StarCraft 2 Red Bull Stratos Red Bull Straight Rhythm Red Bull Double Pipe Red Bull Global Rallycross Championship Red Bull Cliff Diving World Series Red Bull Drifting World Championship Red Bull Rampage Red Bull Road Rage Red Bull King of the Rock Tournament Red Bull Romaniacs Hard Enduro Rallye Red Bull Crashed Ice Red Bull X-Fighters Red Bull X-Alps Red Bull Air Race World Championship Red Bull Flugtag Red Bull Frozen Rush Red Bull Foxhunt Red Bull Cape Fear Red Bull Paperwings Red Bull Training Grounds Red Bull Art of Motion Red Bull BC One Red Bull Cold Rush Red Bull Dolomitenmann - A 4 - man relay competition including trail running , paragliding , kayaking , mountainbiking Red Bull Elements - A 4 - man relay competition including rowing , trail running , paragliding , mountainbiking Red Bull MotoGP Rookies Cup Indianapolis motorcycle Grand Prix Motorcycle Grand Prix of the Americas Argentine motorcycle Grand Prix ( 2014 - 2015 ) Spanish motorcycle Grand Prix Red Bull Storm Chase - A worldwide windsurfing competition in storm conditions with wind speeds of at least force ten Red Bull Supernatural - A `` backcountry boarding competition designed to challenge , then crown the best all - around snowboarder on the planet . '' Red Bull Soapbox Race Red Bull Unleashed - A surfing event at Surf Snowdonia , north Wales Red Bull World Bridge Series Red Bull Drift Shifters - Drifting event held on a pinball inspired street course in Auckland , New Zealand Red Bull Music Academy Red Bull Can You Make It Red Bull Timelaps Locations ( edit ) Red Bull Ring , a motorsport circuit in Spielberg , Styria , Austria . Red Bull Arena ( Leipzig ) , home stadium of German football club RB Leipzig . Red Bull Arena ( New Jersey ) , home stadium of the American soccer club New York Red Bulls . Red Bull Arena ( Salzburg ) , home stadium of Austrian football club FC Red Bull Salzburg . Hangar - 7 ( Salzburg ) , a multifunctional building with a collection of historical airplanes . Red Bull bridge , a recurring distinctly branded footbridge at race events such as the World Enduro Championship and Rally Deutschland . See also ( edit ) Drink portal Felix Baumgartner -- stratospheric parachute jump 2012 References ( edit ) Jump up ^ Boome , Adam ( 31 May 2011 ) . `` Who makes Red Bull '' . Jump up ^ Burt Helm ( 4 January 2005 ) . `` Energy Drinks Build Their Buzz '' . BloombergBusinessweek . BLOOMBERG L.P . Retrieved 2012 - 08 - 19 . Jump up ^ Red Bull ( 2012 ) . `` Company Figures '' . Red Bull . Red Bull . Retrieved 2012 - 08 - 19 . Jump up ^ `` Red Bull the company - Who makes Red Bull ? Red Bull Origin : : Energy Drink : : Red Bull '' . Energydrink.redbull.com . Retrieved 2013 - 11 - 13 . Jump up ^ Roll , Martin ( 2005 - 10 - 17 ) . Asian Brand Strategy : How Asia Builds Strong Brands . Springer . p. 199 . ISBN 9780230513068 . Retrieved 2012 - 09 - 24 . Jump up ^ Cheryl Tay ( 8 April 2011 ) . `` Former F1 Driver David Coulthard Thrilled Shoppers Along Bukit Bintang In Red Bull Racings Show Car Run '' . Va Va Vroom . Cheryl Tay Pte Ltd . Archived from the original on 8 January 2012 . Retrieved 2012 - 07 - 27 . Jump up ^ `` Company '' . Red Bull . Archived from the original on 2013 - 02 - 01 . Retrieved 2011 - 04 - 02 . Jump up ^ Kuehlwein , JP ; Chaefer , Wolf ( Spring 2017 ) . `` How modern prestige brands create meaning through mission and myth '' . Journal of Brand Strategy . Henry Stewart Publications . 5 ( 4 ) . ISSN 2045 - 855X Jump up ^ Kuehlwein , JP . Ueber - Brands and The Art of Myth - Making . www.ueberbrands.com 12 / 2015 Jump up ^ Schaefer , Wolf and Kuehlwein , JP . 2015 . Rethinking Prestige Branding - Secrets of the Ueber - Brands . Kogan Page . pp. 77ff . ISBN 9780749470036 Jump up ^ Lauria , Peter ( 2008 - 04 - 01 ) . `` Pump the Music : Red Bull Eyes Starting Branded Music Label -- NYPost.com '' . New York Post . 101013 NYPost.com Jump up ^ Pennay A , Lubman DI , Miller P ( 2011 ) . `` Combining energy drinks and alcohol '' ( PDF ) . Retrieved 2015 - 08 - 12 . ^ Jump up to : Sanchis - Gomar F , Pareja - Galeano H , Cervellin G , Lippi G , Earnest CP ( 2015 ) . `` Energy drink overconsumption in adolescents : implications for arrhythmias and other cardiovascular events '' . Can J Cardiol. 31 ( 5 ) : 572 -- 5 . doi : 10.1016 / j. cjca. 2014.12. 019 . PMID 25818530 . ^ Jump up to : Petit A , Karila L , Lejoyeux M ( 2015 ) . `` ( Abuse of energy drinks : does it pose a risk ? ) '' . Presse Med. 44 ( 3 ) : 261 -- 70 . doi : 10.1016 / j. lpm. 2014.07. 029 . PMID 25622514 . ^ Jump up to : `` Scientific Opinion on the safety of caffeine Europäische Behörde für Lebensmittelsicherheit '' . www.efsa.europa.eu . Retrieved 2016 - 01 - 02 . ^ Jump up to : McLellan TM , Lieberman HR ( 2012 ) . `` Do energy drinks contain active components other than caffeine ? '' . Nutr Rev. 70 ( 12 ) : 730 -- 44 . doi : 10.1111 / j. 1753 - 4887.2012. 00525. x . PMID 23206286 . Jump up ^ `` Face value Selling energy '' . Economist.com. 2002 - 05 - 09 . Retrieved 2009 - 06 - 22 . Jump up ^ Kerry A. Dolan ( 28 March 2005 ) . `` The Soda With Buzz '' . Forbes . Retrieved 20 June 2015 . ^ Jump up to : Nivatpumin , Chiratas ; Treeraponpichit , Busrin ( 2015 - 12 - 28 ) . `` Red Bull still charging ahead '' . Bangkok Post . Retrieved 30 December 2015 . ^ Jump up to : `` Red Bull GmbH Company History '' . Funding Universe . Retrieved 2007 - 10 - 12 . Jump up ^ Ligaya , Armina ( 2010 - 05 - 12 ) . `` Region abuzz over energy drinks '' . The National . p. Business section , pp. 1 , 6 . Retrieved 2010 - 05 - 13 . Jump up ^ `` The World 's Billionaires '' . Forbes.com. 2008 - 02 - 11 . Retrieved 2009 - 06 - 22 . Jump up ^ `` The World 's Billionaires '' . Forbes.com. 2008 - 02 - 11 . Retrieved 2009 - 06 - 22 . Jump up ^ `` Red Bull Ingredients : : Energy Drink : : Red Bull '' . Red Bull . Retrieved 2012 - 05 - 04 . Jump up ^ `` Caffeine in Red Bull '' . www.caffeineinformer.com . Retrieved 2017 - 12 - 29 . Jump up ^ `` Sweeteners in Red Bull zero sugar ? : : Energy drink sugar free '' . Red Bull . Retrieved 2012 - 09 - 24 . Jump up ^ `` HEALTH Energy drink claims rejected '' . BBC News . 2001 - 01 - 24 . Retrieved 2009 - 06 - 22 . Jump up ^ Mohney , Gillian . `` New Dietary Guidelines : Limit Added Sugars but Enjoy That Coffee '' . ABC News . Retrieved 24 January 2016 . Jump up ^ Healy , Melissa . `` New Dietary Guidelines for Americans : Watch your sugar , but enjoy the eggs and coffee '' . L.A. Times . Retrieved 24 January 2016 . ^ Jump up to : Van Den Eynde F , Van Baelen PC , Portzky M , Audenaert K ( 2008 ) . `` The effects of energy drinks on cognitive performance '' . Tijdschrift voor psychiatrie. 50 ( 5 ) : 273 -- 81 . PMID 18470842 . Jump up ^ Alford , C. ; Cox , H. ; Wescott , R. ( 2001 - 01 - 01 ) . `` The effects of red bull energy drink on human performance and mood '' . Amino Acids. 21 ( 2 ) : 139 -- 150 . doi : 10.1007 / s007260170021 . ISSN 0939 - 4451 . PMID 11665810 . Jump up ^ `` Tijdschrift voor Psychiatrie - Author index : Baelen , P.C. van '' . 2008 - 12 - 21 . Archived from the original on December 21 , 2008 . Retrieved 2016 - 02 - 02 . ^ Jump up to : Mets , Monique A.J. ; Ketzer , Sander ; Blom , Camilla ; van Gerven , Maartje H. ; van Willigenburg , Gitta M. ; Olivier , Berend ; Verster , Joris C. ( 2011 - 04 - 01 ) . `` Positive effects of Red Bull ® Energy Drink on driving performance during prolonged driving '' . Psychopharmacology. 214 ( 3 ) : 737 -- 745 . doi : 10.1007 / s00213 - 010 - 2078 - 2 . ISSN 1432 - 2072 . PMC 3053448 . PMID 21063868 . Jump up ^ Baum , M. ; Weiss , M. ( 2001 - 01 - 01 ) . `` The influence of a taurine containing drink on cardiac parameters before and after exercise measured by echocardiography '' . Amino Acids. 20 ( 1 ) : 75 -- 82 . doi : 10.1007 / s007260170067 . ISSN 0939 - 4451 . PMID 11310932 . Jump up ^ Geiß , K. - R. ; Jester , I. ; Falke , W. ; Hamm , M. ; Waag , K. - L. ( 1994 - 02 - 01 ) . `` The effect of a taurine - containing drink on performance in 10 endurance - athletes '' . Amino Acids. 7 ( 1 ) : 45 -- 56 . doi : 10.1007 / BF00808445 . ISSN 0939 - 4451 . Jump up ^ Forbes , Scott C. ; Candow , Darren G. ; Little , Jonathan P. ; Magnus , Charlene ; Chilibeck , Philip D. ( 2007 - 10 - 01 ) . `` Effect of Red Bull energy drink on repeated Wingate cycle performance and bench - press muscle endurance '' . International Journal of Sport Nutrition and Exercise Metabolism. 17 ( 5 ) : 433 -- 444 . ISSN 1526 - 484X . PMID 18046053 . Jump up ^ EFSA Panel on Food Additives and Nutrient Sources added to Food ( 2009 ) . `` The use of taurine and D - glucurono - γ - lactone as constituents of the so - called ' energy ' drinks '' . The EFSA Journal . 935 : 1 -- 31 . doi : 10.2903 / j. efsa. 2009.935 . Jump up ^ Scientific Opinion on the substantiation of health claims related to taurine and protection of DNA , proteins and lipids from oxidative damage ( ID 612 , 1658 , 1959 ) , energy - yielding metabolism ( ID 614 ) , and delay in the onset of fatigue and enhancement of physical performance ( ID 1660 ) pursuant to Article 13 ( 1 ) of Regulation ( EC ) No 1924 / 2006 , European Food Safety Authority 2009 , http://www.efsa.europa.eu/fr/efsajournal/doc/1260.pdf Jump up ^ Persad , Nikki ; McQueen , Cydney E. ; Shields , Kelly M. ; Clauson , Kevin A. ( 2008 ) . `` Safety issues associated with commercially available energy drinks '' . Journal of the American Pharmacists Association . 48 ( 3 ) : e55 -- 63 ; quiz e64 -- 7 . doi : 10.1331 / JAPhA. 2008.07055 . PMID 18595815 . Jump up ^ `` 1 in 10 Energy Drink - Related Emergency Department Visits Results in Hospitalization '' ( PDF ) . Substance Abuse and Mental Health Services Administration . Retrieved 9 March 2017 . Jump up ^ `` Red Bull caffeine beverage review '' . About.com. 2009 - 10 - 22 . Retrieved 2012 - 05 - 29 . Jump up ^ `` Nutrition Facts and Analysis for Energy drink , RED BULL , with added caffeine , niacin , pantothenic acid , vitamins B6 and B12 '' . Condé Nast . Retrieved 2012 - 05 - 29 . Jump up ^ Owen , Daniel ( 2006 - 01 - 15 ) . `` How much caffeine is there in ( drink / food / pill ) ? '' . Coffeefaq.com . Retrieved 2012 - 09 - 24 . Jump up ^ Alford C , Cox H , Wescott R ( 2001 ) . `` The effects of red bull energy drink on human performance and mood '' . Amino Acids. 21 ( 2 ) : 139 -- 50 . doi : 10.1007 / s007260170021 . PMID 11665810 . Jump up ^ `` It 's Your Health - Caffeine '' . Health Canada . April 13 , 2012 . Retrieved November 24 , 2014 . ^ Jump up to : Winston AP ( 2005 ) . `` Neuropsychiatric effects of caffeine '' . Advances in Psychiatric Treatment . 11 ( 6 ) : 432 -- 439 . doi : 10.1192 / apt. 11.6. 432 . Jump up ^ Warning : Energy Drinks Contain Caffeine by Allison Aubrey . Morning Edition , National Public Radio , 24 September 2008 . ^ Jump up to : Frith , Maxine ( 2004 - 02 - 07 ) . `` European court backs ban on Red Bull over health concerns '' . The Independent . London . Retrieved 2010 - 11 - 22 . Jump up ^ `` France ends 12 - year ban on energy drink Red Bull '' . Paris . Reuters. 15 July 2008 . Retrieved 2010 - 11 - 22 . Jump up ^ Habib Toumi , Bureau Chief ( 2013 - 05 - 16 ) . `` Kuwaiti player dies after taking energy drinks '' . GulfNews . Retrieved 2014 - 07 - 05 . Jump up ^ `` KUNA : Decisions regulating sale of energy drinks issued - Economics - 21 / 10 / 2012 '' . Kuwait News Agency ( KUNA ) . 2012 - 10 - 21 . Retrieved 2014 - 07 - 05 . Jump up ^ كونا : وزارة التجارة تصدر قرارا ينظم انتاج وعرض وبيع مشروبات الطاقة - التجارة - 21 / 10 / 2012 ( in Arabic ) . Kuwait News Agency ( KUNA ) . 2012 - 10 - 21 . Retrieved 2014 - 07 - 05 . Jump up ^ `` التجارة '' : مشروبات الطاقة ممنوعة لمن دون الـ 16 ( in Arabic ) . Al - Qabas newspaper . 2012 - 10 - 21 . Archived from the original on 2014 - 07 - 08 . Retrieved 2014 - 07 - 05 . Jump up ^ `` Red Bull and a crisis in Kuwait that has n't hit the headlines '' . Alex of Arabia 's Blog . Retrieved 2014 - 07 - 05 . Jump up ^ `` Red Bull Kuwait Loses Millions '' . Q8Critic. 2012 - 10 - 24 . Archived from the original on 2013 - 06 - 25 . Retrieved 2013 - 06 - 28 . Jump up ^ http://omarker.com/?p=15727 Jump up ^ `` Red Bull pursues Redwell brewery in Norwich over name '' . BBC News . 2013 - 08 - 14 . Retrieved 2013 - 11 - 13 . Jump up ^ `` Red Bull ends dispute with Redwell brewery in Norwich '' . BBC News . 2013 - 08 - 14 . Retrieved 2013 - 11 - 13 . Jump up ^ `` Red Bull Puts Up $13 M To Settle False Advertising Suit - Law360 '' . www.law360.com . Retrieved 2017 - 03 - 08 . Jump up ^ `` Red Bull Agrees to $13 M False Advertising Class Action Settlement '' . Top Class Actions. 2014 - 08 - 04 . Retrieved 2017 - 03 - 08 . Jump up ^ `` Drink Red Bull ? You may have $10 coming to you '' . cbsnews.com. 2014 - 10 - 09 . Retrieved 2015 - 02 - 16 . Jump up ^ `` So Red Bull does n't actually ' give you wings ' '' . Bbc.co.uk. 2014 - 10 - 09 . Retrieved 2014 - 11 - 03 . Jump up ^ `` Red Bull to Pay $13 Million '' . bevnet.com . Law360 . Retrieved 2014 - 10 - 21 . Jump up ^ Duggan , Oliver ( 2014 - 10 - 11 ) . `` $13 m lawsuit proves Red Bull does n't give you wings '' . telegraph.co.uk . Retrieved 2017 - 10 - 21 . Jump up ^ sean . `` Red Bull Rampage Recap '' . Mountain Bike Action Magazine . Retrieved 2017 - 03 - 08 . Jump up ^ `` Drew Bezanson wins Red Bull Framed Reactions BMX Park comp '' . XGames . Retrieved 2017 - 03 - 08 . Jump up ^ `` 2017 Red Bull KTM Team Intro Transworld Motocross '' . Transworld Motocross. 2016 - 12 - 18 . Retrieved 2017 - 03 - 08 . Jump up ^ ( PWA ) , Professional Windsurfing Association . `` Red Bull Storm Chase '' . PWA World Tour . Retrieved 2017 - 03 - 08 . Jump up ^ `` Red Bull Snowboarding Snowboarder Magazine '' . Snowboarder Magazine . Retrieved 2017 - 03 - 08 . Jump up ^ `` red bull skateboarding TransWorld SKATEboarding '' . TransWorld SKATEboarding . Retrieved 2017 - 03 - 08 . Jump up ^ `` Chasing Niagara Falls in a Kayak VICE Sports '' . VICE Sports . Retrieved 2017 - 03 - 08 . Jump up ^ `` Red Bull High Stakes - Overview '' . www.regattacentral.com . Retrieved 2017 - 03 - 08 . Jump up ^ `` Red Bull Wake of Steel 2016 '' . Wakeboarding Magazine . Retrieved 2017 - 03 - 08 . Jump up ^ `` Red Bull Cliff Diving World Series Announces 2017 Schedule '' . Retrieved 2017 - 03 - 08 . Jump up ^ Kolasa - Sikiaridi , Kerry . `` ... Wins Parkour Red Bull Art of Motion in Santorini ( video ) GreekReporter.com '' . Retrieved 2017 - 03 - 08 . Jump up ^ RED BULL QIANTANG : SURFING SHOOTOUT , retrieved 2017 - 03 - 08 Jump up ^ `` Red Bull Crashed Ice Championship In Saint Paul , MN - Visit Saint Paul '' . www.visitsaintpaul.com . Retrieved 2017 - 03 - 08 . Jump up ^ `` Real - Time Data Tech Dazzles Red Bull X-Fighters Fans '' . iQ by Intel . 2016 - 06 - 22 . Retrieved 2017 - 03 - 08 . Jump up ^ GRC . `` Red Bull Global Rallycross New England set for June '' . www.racer.com . Retrieved 2017 - 03 - 08 . Jump up ^ `` Red Bull `` positive '' RB13 can challenge Mercedes in F1 2017 `` . Motorsport.com . Retrieved 2017 - 03 - 08 . Jump up ^ `` Archrival Red Bull Racing in NASCAR '' . Archrival . Retrieved 2017 - 03 - 08 . Jump up ^ `` Redbull Flying Bach tour brings Bach and breakdancing to Dallas in 2017 '' . AXS . Retrieved 2017 - 03 - 08 . Jump up ^ Pajot , S. ( 2011 - 11 - 03 ) . `` Calling Battle Rappers ! Enter Red Bull EmSee Miami and Win a Trip to the Finals in Atlanta '' . Miami New Times . Retrieved 2017 - 03 - 08 . Jump up ^ `` Red Bull 's high - flying Flugtag competition to return to Nashville '' . The Tennessean . Retrieved 2017 - 03 - 08 . Jump up ^ `` Austria - FC Salzburg - Results , fixtures , squad , statistics , photos , videos and news - Soccerway '' . us.soccerway.com . Retrieved 2017 - 03 - 08 . ^ Jump up to : `` Analysis RB Leipzig 's Bundesliga debut is like a fairy tale in which the bad guys are winning '' . Washington Post . Retrieved 2017 - 03 - 08 . Jump up ^ `` Brazil - Red Bull Brasil - Results , fixtures , squad , statistics , photos , videos and news - Soccerway '' . us.soccerway.com . Retrieved 2017 - 03 - 08 . Jump up ^ `` RED BULL HISTORY '' . speedace.info . Electrick Publications . Retrieved 2012 - 07 - 27 . Jump up ^ `` Red Bull becomes first brand on PlayStation Home '' . Retrieved 2017 - 03 - 08 . Jump up ^ Fahey , Mike . `` Red Bull Gives PlayStation Home New Things To Do '' . Kotaku . Retrieved 2017 - 03 - 08 . Jump up ^ `` Red Bull House of Skate '' . YourPSHome.net . Retrieved 2017 - 03 - 08 . Jump up ^ `` iTunes - Browse the top free apps on the App Store - Apple '' . Apple . Retrieved 2016 - 02 - 11 . Jump up ^ `` GTVault - Gran Turismo 5Setup for '' . www.gtvault.com . Retrieved 2017 - 03 - 08 . Jump up ^ Abbey - Lambertz , Kate ( 2013 - 08 - 07 ) . `` Red Bull House Of Art Bolsters Detroit Scene By Giving Artists On The Verge A Leg Up ( VIDEO ) '' . Huffington Post . Retrieved 2015 - 01 - 24 . Jump up ^ Hammerstein , BJ . `` Detroit artist star of new Red Bull ad '' . Detroit Free Press . Retrieved 2015 - 01 - 24 . Jump up ^ Rigby , Claire . `` Red Bull Station opens up downtown '' . Time Out Brazil . Retrieved 2015 - 01 - 24 . Jump up ^ DeVito , Lee . `` Meet the creative minds behind the Red Bull House of Art 's Cycle 8 '' . Metro Times . Retrieved 24 January 2015 . Jump up ^ Trevino , Jessica J. `` Meet the 6 local artists featured in the Red Bull House of Art exhibit '' . Detroit Free Press . Retrieved 24 January 2015 . Jump up ^ DeVito , Lee . `` Red Bull House of Art '' . MetroTimes . Retrieved 24 January 2015 . Jump up ^ Lacy , Eric . `` Grammys 2014 : Watch Detroit Red Bull House of Art ' City of Soldiers ' commercial aired during show '' . MLive . Retrieved 24 January 2015 . Jump up ^ `` Red Bull New York -- Ball Sports Profiles '' . Red Bull . Retrieved 2012 - 05 - 04 . Jump up ^ `` Holden commits to Supercars Supercars '' . Supercars . Retrieved 2017 - 10 - 15 . Jump up ^ `` OG and the rise of player centred teams '' . redbull.com . Retrieved 2017 - 08 - 02 . Jump up ^ Origen staff ( 2015 - 05 - 14 ) . `` xPeke becomes a Red Bull Athlete '' . Retrieved 2017 - 06 - 30 . Jump up ^ Galaflow ( 2015 - 05 - 15 ) . `` xPeke ist der erste RedBull - Athlet ! '' . Summoners-Inn.de ( in German ) . Retrieved 2017 - 06 - 30 . Jump up ^ `` Reggie Bush Goes to Red Bull -- ESPN The Magazine '' . Sports.espn.go.com . Retrieved 2009 - 06 - 22 . Jump up ^ `` Hikaru Nakamura Always try to win at all costs '' . Jump up ^ `` US # 1 Nakamura Signs with Red Bull '' . Jump up ^ Womack , Barrett ( February 16 , 2016 ) . `` How Leffen Will Take Over the World '' . Red Bull eSports . Jump up ^ `` Daigo signs with Red Bull as Mad Catz officially exits player sponsorship '' . Dot Esports . May 16 , 2016 . Retrieved 2016 - 11 - 26 . Jump up ^ `` Japanese fighting game legend Bonchan joins Red Bull '' . Dot Esports . September 1 , 2015 . Retrieved 2016 - 11 - 26 . Jump up ^ Walker , Ian ( July 22 , 2015 ) . `` Red Bull Gives Darryl `` Snake Eyez '' Lewis Wings `` . Shoryuken.com . Retrieved 2016 - 11 - 26 . Jump up ^ `` Luffy signs with Red Bull '' . Dot Esports . February 14 , 2016 . Retrieved 2016 - 11 - 26 . Jump up ^ `` Breaking Down The Elite Eight : : Red Bull Esports '' . Red Bull . Retrieved 2013 - 06 - 21 . Jump up ^ `` Red Bull Cold Rush 2012 : : Red Bull '' . Red Bull . Retrieved 2012 - 04 - 17 . Jump up ^ `` Red Bull Dolomitenmann `` . Dolomitenmann.com. 2013 - 04 - 03 . Retrieved 2013 - 07 - 28 . Jump up ^ `` Red Bull Elements '' . Red Bull Aventure . Retrieved 2013 - 11 - 13 . Jump up ^ `` Red Bull Storm Chase '' . Red Bull . Retrieved 2014 - 02 - 09 . Jump up ^ `` Red Bull Supernatural Event Web Page '' . Retrieved 2012 - 08 - 22 . Jump up ^ `` Red Bull Soapbox Race '' . Archived from the original on 2013 - 02 - 07 . Retrieved 2013 - 02 - 07 . Jump up ^ Red Bull Unleashed Jump up ^ `` 2014 World Bridge Series - World Bridge Federation '' . Worldbridge.org. 2013 - 07 - 24 . Retrieved 2014 - 06 - 10 . External links ( edit ) Wikimedia Commons has media related to Red Bull . Official website Krating Daeng ( Thai Red Bull ) : www.kratingdaeng.com/main.php # / lets_get_start Red Bull Energy Drink Red Bull 's Total Marketing -- slideshow by Der Spiegel Red Bull GmbH Beverages Red Bull Red Bull Simply Cola Carpe Diem Krating Daeng People Dietrich Mateschitz Chaleo Yoovidhya Chalerm Yoovidhya Events Crashed Ice Dolomitenmann Red Bull Air Race World Championship Red Bull Art of Motion Red Bull Art of the Can Red Bull BC One Red Bull Big Wave Africa Red Bull Cape Fear Red Bull Cliff Diving World Series Red Bull Drifting World Championship Red Bull Flugtag Frozen Rush Red Bull Global Rallycross Swatch Beach Volleyball Major Series Red Bull Illume Red Bull MotoGP Rookies Cup Red Bull Music Academy Red Bull Paper Wings Red Bull Rampage Red Bull Road Rage Red Bull Romaniacs Hard Enduro Rallye Red Bull Stratos Red Bull Trolley Grand Prix Red Bull X-Alps Red Bull X-Fighters Sports teams Association football FC Red Bull Salzburg FC Liefering New York Red Bulls II U-23 RB Leipzig Red Bull Brasil Red Bull Ghana Ice hockey EC Red Bull Salzburg EHC Red Bull München Auto racing Red Bull Racing Scuderia Toro Rosso Red Bull Junior Team Red Bull Racing Australia Red Bull Racing Team Motorcycle racing Red Bull Honda World Superbike Team Red Bull KTM Factory Racing RBR F1 cars RB1 RB2 RB3 RB4 RB5 RB6 RB7 RB8 RB9 RB10 RB11 RB12 RB13 RB14 STR F1 cars STR1 STR2 / STR2B STR3 STR4 STR5 STR6 STR7 STR8 STR9 STR10 STR11 STR12 STR13 Arenas Red Bull Arena ( Leipzig ) Red Bull Arena ( New Jersey ) Red Bull Arena ( Salzburg ) Other The Art of Flight Flying Bulls Aerobatics Team Hangar - 7 New York Red Bulls Academy Red Bull New Year No Limits Red Bull Records Red Bull Ring Red Bull X2010 Red Bulls ServusTV The Red Bulletin Energy drinks Monster Beverage BPM Energy Burn Formula 50 Full Throttle Monster Mother NOS RAC 124 Relentless Tab Energy Vault Von Dutch PepsiCo AMP Energy Josta Mountain Dew Rockstar Sting Energy Drink Independent 51 Fifty Bacchus - F Bawls Carabao Cocaine Coolah Energy Crunk Demon Energy Desperta Ferro ! EJ - 10 Emerge Four Gay Fuel Hell Energy Joose Krating Daeng Lipovitan Lost Energy M - 150 Nae Danger Oronamin Pimp Juice Pussy Red Bull Reload Red Rooster Rip It Semtex Shark Street King Tiger V Venom Other Battery List of energy drinks Streamy Awards winners -- Channel , Series , or Show Overall Action and Sci - Fi H+ : The Digital Series ( 2013 ) Video Game High School ( 2014 ) Corridor Digital ( 2015 ) Day 5 ( 2016 ) Crypt TV ( 2017 ) Animated How It Should Have Ended ( 2010 ) Electric City ( 2013 ) RWBY ( 2014 ) The Cyanide & Happiness Show ( 2015 ) sWooZie ( 2016 ) RWBY ( Volume 4 ) ( 2017 ) Breakout Creator Matthew Santoro ( 2015 ) Liza Koshy ( 2016 ) David Dobrik ( 2017 ) Comedy Liza Koshy ( 2017 ) Documentary ‡ Atari : Game Over ( 2015 ) The Banker Suicides ( 2016 ) Psycho Family ( 2017 ) First Person DailyGrace ( 2013 ) Jenna Marbles ( 2014 ) IISuperwomanII ( 2015 ) Casey Neistat ( 2016 ) Lilly Singh ( 2017 ) Immersive BlackBoxTV ( 2017 ) International OzGirl ( 2010 ) Travel Story ( 2013 ) Noob ( 2014 ) Racka Racka ( 2015 ) HolaSoyGerman ( 2016 ) Juanpa Zurita ( 2017 ) Series Comedy Series The Guild ( 2009 ) Between Two Ferns with Zach Galifianakis ( 2010 ) Burning Love ( 2013 ) My Drunk Kitchen ( 2014 ) Flula ( 2015 ) Good Mythical Morning ( 2016 ) Rhett and Link 's Buddy System ( 2017 ) Drama Series Battlestar Galactica : The Face of the Enemy ( 2009 ) The Bannen Way ( 2010 ) Halo 4 : Forward Unto Dawn ( 2013 ) The Lizzie Bennet Diaries ( 2014 ) BlackBoxTV ( 2015 ) I Ship It ( 2016 ) Cold ( 2017 ) Indie Series Little Horribles ( 2014 ) Eat Our Feelings ( 2015 ) Brooklyn Sound ( 2016 ) Brown Girls ( 2017 ) Non-Fiction Series The Shatner Project ( 2009 ) The Secret Life of Scientists ( 2010 ) Kids React ( 2013 ) Comedians in Cars Getting Coffee ( 2014 ) Good Mythical Morning ( 2015 ) The Try Guys ( 2016 ) Ladylike ( 2017 ) Subject Beauty Missglamorazzi ( 2014 ) Gigi Gorgeous ( 2015 ) Kandee Johnson ( 2016 ) PatrickStarrr ( 2017 ) Fashion Bethany Mota ( 2014 ) Lauren Elizabeth ( 2015 ) Amanda Steele ( 2016 ) Tess Christine ( 2017 ) Food ‡ Epic Meal Time ( 2014 ) Epic Meal Time ( 2015 ) Tiny Kitchen ( 2016 ) Worth It ( 2017 ) Gaming Smosh Games ( 2014 ) PewDiePie ( 2015 ) The Game Theorists ( 2016 ) Smosh Games ( 2017 ) Sports and Wellness † Blogilates ( 2017 ) Kids and Family Kids React ( 2014 ) What 's Up Moms ( 2015 ) Kid President ( 2016 ) Brooklyn & Bailey ( 2017 ) Lifestyle iJustine ( 2015 ) GloZell Green ( 2016 ) LaurDIY ( 2017 ) News and Culture Alive in Baghdad ( 2009 ) Auto - Tune the News ( 2010 ) The Philip DeFranco Show ( 2013 ) SourceFed ( 2014 ) The Young Turks ( 2015 ) The Philip DeFranco Show ( 2016 ) The Young Turks ( 2017 ) Pranks PrankvsPrank ( 2015 ) Roman Atwood ( 2016 ) Science or Education Vsauce ( 2014 ) Vsauce ( 2015 ) Laci Green ( 2016 ) Veritasium ( 2017 ) Discontinued Branded Entertainment Back on Topps ( 2010 ) Leap Year ( 2013 ) Health and Wellness Blogilates ( 2014 ) Blogilates ( 2015 ) Yoga With Adriene ( 2016 ) Hosted EPIC FU ( 2009 ) Diggnation ( 2010 ) Spin - Off The Office : Subtle Sexuality ( 2010 ) The Walking Dead : Cold Storage ( 2013 ) Vandeveon & Mike ( 2014 ) The Tonight Show Starring Jimmy Fallon : Backstage ( 2015 ) Sports Red Bull ( 2014 ) Dude Perfect ( 2015 ) 30 for 30 Shorts ( 2016 ) Other categories Auto - Tune the News ( 2010 -- Experimental ) What 's Trending ( 2013 -- Live ) Epic Rap Battles of History ( 2013 -- Music ) Jabbawockeez ( 2014 -- Dance ) MatPat 's Game Lab ( 2016 -- Virtual Reality and 360 ) † = category was merged from others ‡ = category was renamed Retrieved from `` https://en.wikipedia.org/w/index.php?title=Red_Bull&oldid=822186025 '' Categories : Red Bull Austrian brands Companies of Thailand Energy drinks Products introduced in 1987 Thai brands Hidden categories : All articles with self - published sources Articles with self - published sources from April 2013 CS1 uses Arabic - language script ( ar ) CS1 Arabic - language sources ( ar ) CS1 German - language sources ( de ) Pages using deprecated image syntax Articles with hRecipes Articles with hProducts Articles needing additional references from March 2017 All articles needing additional references All articles with unsourced statements Articles with unsourced statements from August 2015 Articles containing potentially dated statements from 2014 All articles containing potentially dated statements Articles to be split from March 2016 All articles to be split Talk Variants Contents About Wikipedia Asturianu Azərbaycanca Беларуская Български བོད ་ ཡིག Brezhoneg Català Čeština Dansk Deutsch Eesti Ελληνικά Español Euskara فارسی Français Galego 한국어 Hrvatski Bahasa Indonesia Italiano עברית ქართული Latviešu Lietuvių Magyar Nederlands 日本 語 Norsk Norsk nynorsk Oʻzbekcha / ўзбекча Polski Português Română Русский Shqip Simple English Slovenčina Slovenščina Српски / srpski Suomi Svenska Türkçe Українська Tiếng Việt 粵語 中文 Edit links This page was last edited on 24 January 2018 , at 22 : 12 . Text is available under the Creative Commons Attribution - ShareAlike License ; additional terms may apply . By using this site , you agree to the Terms of Use and Privacy Policy . Wikipedia ® is a registered trademark of the Wikimedia Foundation , Inc. , a non-profit organization . About Wikipedia",
 'qas': [{'question': 'when did red bull come to the united states',
   'id': '0'}]}
]

In [73]:
#validation_new = validation
answers, probabilities = model.predict(validation)

print("ans:", answers)
print("prob:", probabilities)

add example index and unique id: 100%|██████████| 2/2 [00:00<00:00, 27413.75it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

ans: [{'id': '14208778017604803798', 'answer': ['empty']}, {'id': '1074881708737617818', 'answer': ['empty']}]
prob: [{'id': '14208778017604803798', 'probability': [3.466326211119808e-07]}, {'id': '1074881708737617818', 'probability': [3.4393510558124425e-07]}]


In [ ]:
print(validation)